In [1]:
import pandas as pd
import numpy as np
import ast

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
import pandas as pd

# ---------------- MOVIES DATA ----------------
movies_data = {
    "movie_id": [1, 2, 3, 4, 5, 6],
    "title": [
        "Avatar",
        "Titanic",
        "Inception",
        "The Dark Knight",
        "Interstellar",
        "The Avengers"
    ],
    "overview": [
        "A marine on an alien planet",
        "A love story on a sinking ship",
        "A thief who steals information through dreams",
        "Batman fights crime in Gotham City",
        "A team travels through space and time",
        "Superheroes save the world from destruction"
    ],
    "genres": [
        "[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}]",
        "[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",
        "[{'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}]",
        "[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}]",
        "[{'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}]",
        "[{'id': 28, 'name': 'Action'}, {'id': 14, 'name': 'Fantasy'}]"
    ],
    "keywords": [
        "[{'id': 1, 'name': 'future'}]",
        "[{'id': 2, 'name': 'ship'}]",
        "[{'id': 3, 'name': 'dream'}]",
        "[{'id': 4, 'name': 'hero'}]",
        "[{'id': 5, 'name': 'space'}]",
        "[{'id': 6, 'name': 'superhero'}]"
    ]
}

# ---------------- CREDITS DATA ----------------
credits_data = {
    "movie_id": [1, 2, 3, 4, 5, 6],
    "title": [
        "Avatar",
        "Titanic",
        "Inception",
        "The Dark Knight",
        "Interstellar",
        "The Avengers"
    ],
    "cast": [
        "[{'name': 'Sam Worthington'}, {'name': 'Zoe Saldana'}]",
        "[{'name': 'Leonardo DiCaprio'}, {'name': 'Kate Winslet'}]",
        "[{'name': 'Leonardo DiCaprio'}, {'name': 'Joseph Gordon-Levitt'}]",
        "[{'name': 'Christian Bale'}, {'name': 'Heath Ledger'}]",
        "[{'name': 'Matthew McConaughey'}, {'name': 'Anne Hathaway'}]",
        "[{'name': 'Robert Downey Jr.'}, {'name': 'Chris Evans'}]"
    ],
    "crew": [
        "[{'job': 'Director', 'name': 'James Cameron'}]",
        "[{'job': 'Director', 'name': 'James Cameron'}]",
        "[{'job': 'Director', 'name': 'Christopher Nolan'}]",
        "[{'job': 'Director', 'name': 'Christopher Nolan'}]",
        "[{'job': 'Director', 'name': 'Christopher Nolan'}]",
        "[{'job': 'Director', 'name': 'Joss Whedon'}]"
    ]
}

# Create DataFrames
movies_df = pd.DataFrame(movies_data)
credits_df = pd.DataFrame(credits_data)

# Save locally
movies_df.to_csv("tmdb_5000_movies.csv", index=False)
credits_df.to_csv("tmdb_5000_credits.csv", index=False)

print("✅ Dataset created successfully")


✅ Dataset created successfully


In [9]:
import pandas as pd

movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")


In [10]:
movies = movies.merge(credits, on='title')


In [11]:
movies.shape
movies.info()
movies.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_id_x  6 non-null      int64 
 1   title       6 non-null      object
 2   overview    6 non-null      object
 3   genres      6 non-null      object
 4   keywords    6 non-null      object
 5   movie_id_y  6 non-null      int64 
 6   cast        6 non-null      object
 7   crew        6 non-null      object
dtypes: int64(2), object(6)
memory usage: 516.0+ bytes


movie_id_x    0
title         0
overview      0
genres        0
keywords      0
movie_id_y    0
cast          0
crew          0
dtype: int64

In [13]:
movies = movies[['title','overview','genres','keywords','cast','crew']]


In [14]:
movies.dropna(inplace=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\983224553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace=True)


In [15]:
def convert(text):
    return [i['name'] for i in ast.literal_eval(text)]


In [16]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\3658858981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres'] = movies['genres'].apply(convert)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\3658858981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['keywords'] = movies['keywords'].apply(convert)


In [17]:
def get_cast(text):
    cast = []
    for i in ast.literal_eval(text)[:3]:
        cast.append(i['name'])
    return cast

movies['cast'] = movies['cast'].apply(get_cast)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\9550432.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['cast'] = movies['cast'].apply(get_cast)


In [18]:
def get_director(text):
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            return i['name']
    return ''

movies['crew'] = movies['crew'].apply(get_director)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\2613691895.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['crew'] = movies['crew'].apply(get_director)


In [19]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: x.replace(" ",""))


C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\2891951727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\2891951727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\2891951727.py:3: SettingWithCopyWarning: 
A value is trying to be s

In [20]:
movies['tags'] = (
    movies['overview'] +
    movies['genres'].apply(lambda x: " ".join(x)) +
    movies['keywords'].apply(lambda x: " ".join(x)) +
    movies['cast'].apply(lambda x: " ".join(x)) +
    movies['crew']
)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\3019051256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = (


In [22]:
new_df = movies[['title','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\375815999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [23]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()


In [24]:
similarity = cosine_similarity(vectors)


In [25]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    
    movies_list = sorted(
        list(enumerate(distances)),
        reverse=True,
        key=lambda x: x[1]
    )[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [27]:
recommend("Titanic")


Avatar
Inception
The Dark Knight
Interstellar
The Avengers


In [28]:
import pickle
pickle.dump(new_df, open("movies.pkl","wb"))
pickle.dump(similarity, open("similarity.pkl","wb"))


In [29]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend', methods=['GET'])
def recommend_movie():
    movie = request.args.get('movie')
    return jsonify(recommend(movie))
